In [1]:
import os
import numpy as np 
import pandas as pd 

In [ ]:
!wget -P ./../../../dataset/ -nc https://cleanlab-public.s3.amazonaws.com/RegressionBenchmark/telomere_ts.csv

In [3]:
path = "./../../../dataset/telomere_ts.csv"
dataset_name = os.path.splitext(os.path.basename(path))[0]
data = pd.read_csv(path, index_col=0)
print("shape of data: ", data.shape)
data.head() 

shape of data:  (10000, 5)


,true_error,true.dna.scg,true.telo.var,true_label,given_label
0,0,8.577178,1.033903,1.039223,1.036155
1,0,10.425544,0.992451,0.997558,0.978202
2,0,10.635183,1.008781,1.013972,0.993370
3,0,8.879612,0.957411,0.962338,0.943759
4,0,8.319504,1.007479,1.012663,1.003617


In [4]:
training_data = data.drop(['true_label', 'true_error'], axis=1)
training_data.head(2)

,true.dna.scg,true.telo.var,given_label
0,8.577178,1.033903,1.036155
1,10.425544,0.992451,0.978202


In [5]:
y = training_data['given_label']
X = training_data.drop(['given_label'], axis=1)

###### Add your training code here ###############
# model = 
# model_fit = model.fit(X,y)
# predictions = model.predict(X)

##### saving the predictions ####################
# savepath = "./../predictions/model_predictions.npy"
# np.save(savepath, np.array(predictions))

## Section section is relevant only if using AutoGluon

Uncomment the code in following cells and run to generate output and save to predictions folder. 

### set up arguments 

In [6]:
from autogluon.tabular import TabularPredictor

data_without_label = training_data.drop(['given_label'], axis=1)
savepath = "./../trained_models/"
if not os.path.isdir(savepath):
    os.mkdir(savepath)

predictorArgs = {
                    "path"             : savepath, 
                    "label"            : 'given_label',
                    "eval_metric"      : 'r2',
                    "problem_type"     : 'regression'
                }

hyperparameters = { 
                    'GBM'   : {}, 
                    'FASTAI': {}, 
                    'RF'    : {'criterion': 'squared_error', 
                                'ag_args': {'name_suffix': 'MSE', 
                                'problem_types': ['regression', 'quantile']}}
                    }

### Normal Model fitting

In [7]:
predictor = TabularPredictor(**predictorArgs)
predictor.fit(training_data, hyperparameters=hyperparameters)
leaderboard = predictor.leaderboard(training_data, silent=True)
leaderboard

Beginning AutoGluon training ...
AutoGluon will save models to "./../trained_models/"
AutoGluon Version:  0.5.2
Python Version:     3.9.15
Operating System:   Darwin
Train Data Rows:    10000
Train Data Columns: 2
Label Column: given_label
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    2574.3 MB
	Train Data (Original)  Memory Usage: 0.16 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 2 | ['true.dna.scg', 'true.telo.var']
	Types of features in processe

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestMSE,0.947820,0.768106,0.221244,0.062303,1.304385,0.221244,0.062303,1.304385,1,True,2
1,NeuralNetFastAI,0.803555,0.802743,0.055756,0.008593,6.149800,0.055756,0.008593,6.149800,1,True,3
2,WeightedEnsemble_L2,0.803555,0.802743,0.074996,0.008750,6.178476,0.019240,0.000157,0.028676,2,True,4
3,LightGBM,0.078406,0.077764,0.001782,0.000672,1.893509,0.001782,0.000672,1.893509,1,True,1


### Cross Validation fitting

In [8]:
predictor_cv = TabularPredictor(**predictorArgs) 
predictor_cv.fit(training_data, presets="best_quality", num_stack_levels= 0, hyperparameters=hyperparameters)
leaderboard_cv = predictor_cv.leaderboard(training_data, silent=True)
leaderboard_cv

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "./../trained_models/"
AutoGluon Version:  0.5.2
Python Version:     3.9.15
Operating System:   Darwin
Train Data Rows:    10000
Train Data Columns: 2
Label Column: given_label
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    2459.3 MB
	Train Data (Original)  Memory Usage: 0.16 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in or

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestMSE_BAG_L1,0.968345,0.767919,0.450438,0.314851,1.186730,0.450438,0.314851,1.186730,1,True,2
1,WeightedEnsemble_L2,0.808825,0.803633,0.997547,0.380238,42.479214,0.001784,0.000203,0.060487,2,True,4
2,NeuralNetFastAI_BAG_L1,0.804190,0.803626,0.545325,0.065183,41.231997,0.545325,0.065183,41.231997,1,True,3
3,LightGBM_BAG_L1,0.078439,0.077949,0.011459,0.007309,11.233089,0.011459,0.007309,11.233089,1,True,1


### predicting and saving arrays 

In [ ]:
models = {  'normal'            : list(leaderboard['model']), 
            'crossValidation'   : list(np.append(leaderboard_cv['model'], ['oof']))}

predictors = { 'normal'          : predictor, 
               'crossValidation' : predictor_cv}

# check if path is available, else create it. 
pred_path = "./../predictions/"
if not os.path.isdir(pred_path):
    os.mkdir(pred_path)

for model_type in models.keys():
    if model_type == "normal": 
        for model in models[model_type]:
            savepath = os.path.join(pred_path, model+".npy")
            predictions = predictors[model_type].predict(data_without_label, model=model)
            np.save(savepath, np.array(predictions))
    elif model_type == "crossValidation":
        for model in models[model_type]:
            if model == "oof":
                predictions = predictors[model_type].get_oof_pred()
                savepath = os.path.join(pred_path, model+".npy")
            elif model in models['normal']:
                predictions = predictors[model_type].predict(data_without_label, model=model)
                savepath = os.path.join(pred_path, model+"_CV.npy")
            else: 
                predictions = predictors[model_type].predict(data_without_label, model=model)
                savepath = os.path.join(pred_path, model+".npy")

            np.save(savepath, np.array(predictions))